In [1]:
import pandas as pd
import numpy as np
import calendar
import requests_cache
import openmeteo_requests
from retry_requests import retry
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [35]:
def predict_solar_weather_m12(lat, lon):
    """
    Fonction intégrée qui :
    1. Récupère les données météo historiques à partir de coordonnées géographiques
    2. Entraîne un modèle de ML sur ces données
    3. Retourne les prédictions météo sur les 12 prochains mois (M+12)
    
    Args:
        lat (float): Latitude du point géographique
        lon (float): Longitude du point géographique
    
    Returns:
        pd.DataFrame: Prédictions météo pour les 12 prochains mois avec colonnes :
                     - date
                     - Température moyenne (°c)
                     - Durée d'ensoleillement moyenne (heure)
                     - GHI moyen (W/m²)
                     - Énergie solaire produite (kWh/m²)
    """
    
    # === ÉTAPE 1: Récupération des données historiques === #

    # Initialisation du client Open-Meteo avec un cache pour ne pas refaire les mêmes requêtes et une logique de retry
    cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    openmeteo = openmeteo_requests.Client(session=retry_session)

    # Liste pour stocker les DataFrames de chaque année
    all_dataframes = []

    # Boucle sur les années de 2015 à 2025 pour récupérer les données
    for year in range(2015, 2026):
        # Préparation de la requête API Open-Meteo pour l’année en cours
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": f"{year}-01-01", 
            "end_date": f"{year}-12-31" if year < 2025 else "2025-07-23",
            "daily": ["sunshine_duration", "temperature_2m_mean"],
            "hourly": "shortwave_radiation",
            "timezone": "Europe/Berlin"
        }

        try:
            # Appel API
            responses = openmeteo.weather_api(url, params=params)
            response = responses[0]

            # --- Traitement des données horaires (irradiance solaire) --- #
            hourly = response.Hourly()
            hourly_shortwave_radiation = hourly.Variables(0).ValuesAsNumpy()

            # Construction du DataFrame horaire
            hourly_data = {"date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            )}
            hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
            hourly_dataframe = pd.DataFrame(data=hourly_data)

            # Ajout des colonnes temporelles
            hourly_dataframe['month'] = hourly_dataframe['date'].dt.month_name(locale='English')
            hourly_dataframe['month_num'] = hourly_dataframe['date'].dt.month
            hourly_dataframe['year'] = hourly_dataframe['date'].dt.year
            hourly_dataframe = hourly_dataframe.iloc[2:]  # Supprimer les 2 premières valeurs erratiques

            # Agrégation mensuelle de l’irradiance (GHI)
            df_ghi = hourly_dataframe.groupby(['year', 'month', 'month_num']).agg(
                ghi_mean_W_m2=('shortwave_radiation', 'mean')
            ).sort_values(by=['year', 'month_num'])

            # --- Traitement des données journalières (température & ensoleillement) --- #
            daily = response.Daily()
            daily_sunshine_duration = daily.Variables(0).ValuesAsNumpy()
            daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()

            # Construction du DataFrame journalier
            daily_data = {"date": pd.date_range(
                start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=daily.Interval()),
                inclusive="left"
            )}
            daily_data["sunshine_duration"] = daily_sunshine_duration
            daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
            daily_dataframe = pd.DataFrame(data=daily_data)

            # Conversion de l’ensoleillement en heures et nettoyage
            daily_dataframe['sunshine_duration_hours'] = daily_dataframe['sunshine_duration'] / 3600
            daily_dataframe.drop(columns='sunshine_duration', inplace=True)
            daily_dataframe = daily_dataframe.iloc[1:]

            # Ajout des colonnes temporelles
            daily_dataframe['month'] = daily_dataframe['date'].dt.month_name(locale='English')
            daily_dataframe['month_num'] = daily_dataframe['date'].dt.month
            daily_dataframe['year'] = daily_dataframe['date'].dt.year
            daily_dataframe['days_in_month'] = daily_dataframe['date'].dt.daysinmonth

            # Agrégation mensuelle de la température et de la durée d’ensoleillement
            df_weather = daily_dataframe.groupby(['year', 'month', 'month_num']).agg(
                temp_mean_c=('temperature_2m_mean', 'mean'),
                sunshine_duration_mean_hour=('sunshine_duration_hours', 'mean'),
                days_in_month=('days_in_month', 'first')
            ).sort_values(by=['year', 'month_num'])

            # Fusion des deux sources (journalier & horaire) pour l’année en cours
            df_year = pd.merge(df_weather, df_ghi, on=['year', 'month', 'month_num'])
            all_dataframes.append(df_year)

        except Exception as e:
            print(f"Erreur lors de la récupération des données pour {year}: {e}")
            continue

    # Vérification si on a bien récupéré des données
    if not all_dataframes:
        print("Aucune donnée récupérée")
        return pd.DataFrame()

    # Concaténation de toutes les années
    df = pd.concat(all_dataframes, ignore_index=False).sort_values(by=['year', 'month_num'])
    df = df.reset_index()

    # === Calcul de la production photovoltaïque historique === #

    # Définition des constantes physiques et techniques du panneau
    EFFICIENCY_PANEL = 0.20
    ANGLE_ORIENTATION_FACTOR = 1.00
    TEMP_COEFFICIENT_PMAX = 0.004
    SYSTEM_LOSS_FACTOR = 0.85
    DELTA_TEMP_PANEL = 20

    # Calcul de l’énergie journalière en kWh/m²/jour
    df['ghi_journalier_kWh_m2_jour'] = (df['ghi_mean_W_m2'] * df['sunshine_duration_mean_hour']) / 1000

    # Température estimée du panneau
    df['panel_temp_c'] = df['temp_mean_c'] + DELTA_TEMP_PANEL

    # Correction de performance selon la température
    df['correction_temp'] = 1 - (df['panel_temp_c'] - 25) * TEMP_COEFFICIENT_PMAX
    df['correction_temp'] = df['correction_temp'].clip(lower=0.95, upper=1.05)

    # Calcul de la production d'énergie journalière
    df['energie_jour_kWh/j/m2'] = (
        df['ghi_journalier_kWh_m2_jour'] *
        EFFICIENCY_PANEL *
        ANGLE_ORIENTATION_FACTOR *
        df['correction_temp'] *
        SYSTEM_LOSS_FACTOR
    ).round(2)

    # Multiplication par le nombre de jours pour avoir la production mensuelle
    df['energie_mois_kWh/mois/m2'] = (df['energie_jour_kWh/j/m2'] * df['days_in_month']).round(2)

    # === ÉTAPE 2: Entraînement du modèle de Machine Learning === #

    # Suppression des colonnes qui introduiraient une fuite de données
    cols_to_drop = ['panel_temp_c', 'correction_temp', 'energie_jour_kWh/j/m2', 'energie_mois_kWh/mois/m2']
    df_ml = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

    # Création de la colonne datetime pour le suivi temporel
    df_ml['date'] = pd.to_datetime(df_ml[['year', 'month_num']].rename(columns={'month_num': 'month'}).assign(day=1))

    # Création des features cycliques (saisonnalité mensuelle)
    df_ml['month_sin'] = np.sin(2 * np.pi * df_ml['month_num'] / 12)
    df_ml['month_cos'] = np.cos(2 * np.pi * df_ml['month_num'] / 12)

    # Définition des features (X) et des cibles (y)
    features = ['year', 'days_in_month', 'month_sin', 'month_cos']
    targets = ['temp_mean_c', 'sunshine_duration_mean_hour', 'ghi_mean_W_m2']

    X = df_ml[features]
    y = df_ml[targets]

    # Séparation temporelle train/test
    train_mask = df_ml['year'] <= 2021
    test_mask = df_ml['year'] > 2021
    X_train, X_test = X[train_mask], X[test_mask]
    y_train, y_test = y[train_mask], y[test_mask]

    # Standardisation des features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Définition du modèle multi-sortie
    base_model = RandomForestRegressor(random_state=42)
    multi_output_model = MultiOutputRegressor(base_model)

    # Définition de la grille de recherche d’hyperparamètres
    param_grid = {
        "estimator__n_estimators": [50, 100, 200],
        "estimator__max_depth": [5, 10, None],
        "estimator__min_samples_split": [2, 5],
    }

    # Utilisation de TimeSeriesSplit pour validation temporelle
    tscv = TimeSeriesSplit(n_splits=5)

    # Recherche des meilleurs hyperparamètres
    grid_search = GridSearchCV(
        estimator=multi_output_model,
        param_grid=param_grid,
        cv=tscv,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=0,
    )
    grid_search.fit(X_train_scaled, y_train)

    # Création du modèle final avec les meilleurs paramètres
    best_params = grid_search.best_params_
    final_model = MultiOutputRegressor(
        RandomForestRegressor(
            n_estimators=best_params['estimator__n_estimators'],
            max_depth=best_params['estimator__max_depth'],
            min_samples_split=best_params['estimator__min_samples_split'],
            random_state=42
        )
    )
    final_model.fit(X_train_scaled, y_train)

    # === ÉTAPE 3: Génération des prédictions sur les 12 mois futurs === #

    # Création d’un calendrier des 12 mois futurs (de août 2025 à juillet 2026)
    future_dates = pd.date_range(start='2025-08-01', periods=12, freq='MS')
    
    # Préparation des features pour prédiction
    future_df = pd.DataFrame({
        'year': future_dates.year,
        'month_num': future_dates.month,
    })
    future_df['days_in_month'] = future_df.apply(
        lambda row: calendar.monthrange(row['year'], row['month_num'])[1], 
        axis=1
    )
    future_df['month_sin'] = np.sin(2 * np.pi * future_df['month_num'] / 12)
    future_df['month_cos'] = np.cos(2 * np.pi * future_df['month_num'] / 12)

    # Standardisation des données futures
    X_future = future_df[features]
    X_future_scaled = scaler.transform(X_future)

    # Prédictions avec le modèle
    y_future_pred = final_model.predict(X_future_scaled)

    # Construction du DataFrame final des prédictions
    predictions_future = pd.DataFrame(y_future_pred, columns=targets)
    predictions_future['date'] = future_dates
    predictions_future['days_in_month'] = future_df['days_in_month'].values

    # === ÉTAPE 4: Estimation de la production solaire future === #

    # Calculs identiques à l’historique pour estimer la production d'énergie
    predictions_future['ghi_journalier_kWh_m2_jour'] = (
        predictions_future['ghi_mean_W_m2'] * predictions_future['sunshine_duration_mean_hour']
    ) / 1000
    predictions_future['panel_temp_c'] = predictions_future['temp_mean_c'] + DELTA_TEMP_PANEL
    predictions_future['correction_temp'] = 1 - (predictions_future['panel_temp_c'] - 25) * TEMP_COEFFICIENT_PMAX
    predictions_future['correction_temp'] = predictions_future['correction_temp'].clip(lower=0.95, upper=1.05)
    predictions_future['energie_jour_kWh/j/m2'] = (
        predictions_future['ghi_journalier_kWh_m2_jour'] *
        EFFICIENCY_PANEL *
        ANGLE_ORIENTATION_FACTOR *
        predictions_future['correction_temp'] *
        SYSTEM_LOSS_FACTOR
    ).round(2)
    predictions_future['energie_mois_kWh/mois/m2'] = (
        predictions_future['energie_jour_kWh/j/m2'] * predictions_future['days_in_month']
    ).round(2)

    # Renommage des colonnes pour un rendu final clair
    predictions_future.rename(columns={
        'temp_mean_c': 'Température moyenne (°c)',
        'sunshine_duration_mean_hour': "Durée d'ensoleillement moyenne (heure)",
        'ghi_mean_W_m2': 'GHI moyen (W/m²)',
        'energie_mois_kWh/mois/m2': 'Énergie solaire produite (kWh/m²)'
    }, inplace=True)
    predictions_future['date'] = predictions_future['date'].dt.to_period('M')

    # Réorganisation des colonnes pour affichage
    final_columns = [
        'date', 
        'Température moyenne (°c)', 
        "Durée d'ensoleillement moyenne (heure)", 
        'GHI moyen (W/m²)',
        'Énergie solaire produite (kWh/m²)'
    ]
    predictions_future = predictions_future[final_columns]

    return predictions_future


In [ ]:
lat, lon = 45.764043, 4.835659 
predictions = predict_solar_weather_m12(lat, lon)
predictions

,date,Température moyenne (°c),Durée d'ensoleillement moyenne (heure),GHI moyen (W/m²),Énergie solaire produite (kWh/m²)
0,2025-08,20.822966,11.472874,229.066043,13.02
1,2025-09,18.632591,9.705785,176.911969,8.40
2,2025-10,12.736322,7.312316,108.024711,4.03
3,2025-11,6.914180,5.180249,62.849457,1.50
4,2025-12,5.428989,4.207250,42.787157,0.93
5,2026-01,4.944563,4.500849,51.707554,1.24
6,2026-02,5.496464,7.431516,103.470726,3.64
7,2026-03,8.491314,8.836445,159.558395,7.44
8,2026-04,12.097798,10.009712,217.996713,10.80
9,2026-05,15.076487,10.369828,229.361898,12.09


: 